# Data Engineering
## Authors: Jonathan Gat and Ivann Schlosser

In [195]:
# importing libraries
import pandas as pd
import numpy as np
import validators
import re
from sqlalchemy import create_engine

## Importing Data

In [196]:
# reading JSON file
rawJson = pd.read_json("Artworks.json", encoding='utf-8')

## Exploring Features

In [197]:
# shape
rawJson.shape

(138990, 28)

In [198]:
# what column does the dataframe contain?
rawJson.columns

Index(['Title', 'Artist', 'ConstituentID', 'ArtistBio', 'Nationality',
       'BeginDate', 'EndDate', 'Gender', 'Date', 'Medium', 'Dimensions',
       'CreditLine', 'AccessionNumber', 'Classification', 'Department',
       'DateAcquired', 'Cataloged', 'ObjectID', 'URL', 'ThumbnailURL',
       'Height (cm)', 'Width (cm)', 'Depth (cm)', 'Weight (kg)',
       'Diameter (cm)', 'Length (cm)', 'Circumference (cm)',
       'Duration (sec.)'],
      dtype='object')

In [199]:
# how many artworks are curator approved?
rawJson['Cataloged'].value_counts()

Y    84249
N    54741
Name: Cataloged, dtype: int64

### Are there any invalid nationality values?
As we can see in the next cell, the following nationality values should be removed from our data set:
- '' (empty string) - 1677 occurrences
- Nationality unknown - 1280 occurrences

In [200]:
# Nationalities
nationalityCheck = rawJson['Nationality'].explode().value_counts()
print('Number of unique values: ',len(nationalityCheck))
print(nationalityCheck.to_string())

Number of unique values:  123
American               65382
French                 24224
German                 10546
British                 6893
Russian                 3526
Italian                 3433
Spanish                 3240
Japanese                2947
Swiss                   2629
Dutch                   1957
Argentine               1919
                        1677
Belgian                 1533
Mexican                 1443
Nationality unknown     1280
Austrian                1249
Canadian                1046
Brazilian                888
Hungarian                817
Czech                    812
Colombian                791
Chilean                  625
Polish                   589
Danish                   588
Venezuelan               521
Ivorian                  483
South African            459
Israeli                  375
Chinese                  348
Swedish                  348
Australian               279
Cuban                    245
Finnish                  242
Indian       

In [201]:
# removing invalid nationality values
rawJson['NationalityClean'] = rawJson['Nationality'].apply(
    lambda row: [val for val in row if val not in ['','Nationality unknown']]   
)

In [202]:
# Did it work?
nationalityCheck = rawJson['NationalityClean'].explode().value_counts()
print('Number of unique values: ',len(nationalityCheck))
print(nationalityCheck.to_string())

Number of unique values:  121
American           65382
French             24224
German             10546
British             6893
Russian             3526
Italian             3433
Spanish             3240
Japanese            2947
Swiss               2629
Dutch               1957
Argentine           1919
Belgian             1533
Mexican             1443
Austrian            1249
Canadian            1046
Brazilian            888
Hungarian            817
Czech                812
Colombian            791
Chilean              625
Polish               589
Danish               588
Venezuelan           521
Ivorian              483
South African        459
Israeli              375
Chinese              348
Swedish              348
Australian           279
Cuban                245
Finnish              242
Indian               213
Norwegian            203
Croatian             182
Portuguese           175
Yugoslav             168
Ukrainian            160
Uruguayan            158
Peruvian            

### Amending nationality values
Some nationality values do not match any of the nationality values in the `countries` dataset and need to be amended. We address this issue by:
1. Loading the `countries` data set into the notebook, eliminating nationality values that occur more than once
2. Locating all of the nationality values within the artworks (`rawJson`) data set that do not match any of values in the `countries` data set
3. Creating a a new dictionary, each element holds the old nationality value as a key and the correspondent nationality as a value
4. Iterating over each row in the `rawJson` data frame and replace any occurrences of the dictionary's keys with the dictionary's values 

#### Step 1: Loading an cleaning the countries data set

In [203]:
# reading the country csv file
countries_header = ['countryName',
                   'countryCode',
                   'capitalName',
                   'capitalLat',
                   'capitalLong',
                    'continentName',
                   'nationality',
                   'nationality2',
                   'nationality3',
                   'nationality4']

countries = pd.read_csv('bubbleMap.csv',
                        header = None,
                        skiprows = 1,
                        names = countries_header)

countries.head()

,countryName,countryCode,capitalName,capitalLat,capitalLong,continentName,nationality,nationality2,nationality3,nationality4
0,Afghanistan,AF,Kabul,34.516667,69.183333,Asia,Afghan,NaN,NaN,NaN
1,Aland Islands,AX,Mariehamn,60.116667,19.900000,Europe,Åland Island,NaN,NaN,NaN
2,Albania,AL,Tirana,41.316667,19.816667,Europe,Albanian,NaN,NaN,NaN
3,Algeria,DZ,Algiers,36.750000,3.050000,Africa,Algerian,NaN,NaN,NaN
4,American Samoa,AS,Pago Pago,-14.266667,-170.700000,Australia,American Samoan,NaN,NaN,NaN


Since nationality / country is a one-to-one relationship, the nationality value must be unique.

In [204]:
# counting the nationality values
nationalityCount = countries['nationality'].value_counts()
nationalityCount = nationalityCount[nationalityCount>1]
print(nationalityCount)
 

American          3
Cypriot           2
Channel Island    2
Somali            2
Dominican         2
Micronesian       2
Name: nationality, dtype: int64


In [205]:
# which countries are correspondent with these values?
countries[countries['nationality'].isin(nationalityCount.index)]

,countryName,countryCode,capitalName,capitalLat,capitalLong,continentName,nationality,nationality2,nationality3,nationality4
55,Cyprus,CY,Nicosia,35.166667,33.366667,Europe,Cypriot,NaN,NaN,NaN
60,Dominica,DM,Roseau,15.300000,-61.400000,North America,Dominican,NaN,NaN,NaN
61,Dominican Republic,DO,Santo Domingo,18.466667,-69.900000,North America,Dominican,NaN,NaN,NaN
71,Federated States of Micronesia,FM,Palikir,6.916667,158.150000,Australia,Micronesian,NaN,NaN,NaN
88,Guinea,GN,Conakry,9.500000,-13.700000,Africa,Channel Island,NaN,NaN,NaN
107,Jersey,JE,Saint Helier,49.183333,-2.100000,Europe,Channel Island,NaN,NaN,NaN
138,Mongolia,MN,Ulaanbaatar,47.916667,106.916667,Asia,Micronesian,NaN,NaN,NaN
156,Northern Cyprus,NaN,North Nicosia,35.183333,33.366667,Europe,Cypriot,NaN,NaN,NaN
197,Somalia,SO,Mogadishu,2.066667,45.333333,Africa,Somali,Somalian,NaN,NaN
198,Somaliland,NaN,Hargeisa,9.550000,44.050000,Africa,Somali,Somalian,NaN,NaN


Unfortunately, these records should be amended manually

In [206]:
# careate a list of amendments
amendments = ['Cypriot',
             'Dominican (Dominica)',
             'Dominican (Dominican Republic)',
             'Micronesian',
             'Guinean (Guinea)',
             'British (Jersey)',
             'Mongolian',
             'Cypriot (Northern Cyprus)',
             'Somalian',
             'Somalian (Somaliland)',
             'American',
             'American (US Minor Outlying Islands)',
             'American (US Virgin Islands)']

# create a dictionary of indexes and amendments
amendmentsDic = dict(zip(countries[countries['nationality'].isin(nationalityCount.index)].index.to_list(), amendments))

# print and check that everything is ok
print(amendmentsDic)

{55: 'Cypriot', 60: 'Dominican (Dominica)', 61: 'Dominican (Dominican Republic)', 71: 'Micronesian', 88: 'Guinean (Guinea)', 107: 'British (Jersey)', 138: 'Mongolian', 156: 'Cypriot (Northern Cyprus)', 197: 'Somalian', 198: 'Somalian (Somaliland)', 231: 'American', 233: 'American (US Minor Outlying Islands)', 234: 'American (US Virgin Islands)'}


In [207]:
# replace invalid nationality values
for i in amendmentsDic:
    countries.at[i,'nationality'] = amendmentsDic[i]

In [208]:
# did it work?
nationalityCount = countries['nationality'].value_counts()
print(nationalityCount[nationalityCount>1])

Series([], Name: nationality, dtype: int64)


In [228]:
# Manual amendments
countries.at[123,'nationality'] = 'Luxembourgish'
countries.at[149,'nationality'] = 'New Zealander'
countries.at[29,'nationality'] = 'Brazilian'
countries.at[30,'nationality'] = 'British (British Indian Ocean Territory)'
countries.at[31,'nationality'] = 'British (British Virgin Islands)'
countries.at[36,'nationality'] = 'Cambodian' 
countries.at[37,'nationality'] = 'Cameroonian'
countries.at[38,'nationality'] = 'Canadian' 
countries.at[39,'nationality'] = 'Cape Verdean' 
countries.at[173,'nationality'] = 'Congolese (Republic of Congo)'

#### Step 2: Identifying inconsistencies

In [230]:
# create a list of nationalities from the countries data set
countriesNationalityList = countries['nationality'].to_list()

# create a list of nationalities from the rawJson data set
rawJsonNationalityList = rawJson['NationalityClean'].explode().unique()

# locate inconsistencies
inconsistencies = [i for i in rawJsonNationalityList if i not in countriesNationalityList]

# remove the first element (0)
inconsistencies.remove(inconsistencies[0])

inconsistencies

['Coptic',
 'Persian',
 'Yugoslav',
 'Scottish',
 'Czechoslovakian',
 'Korean',
 'Bosnian',
 'Canadian Inuit',
 'Tajik',
 'Native American',
 'Welsh',
 'Catalan',
 'Filipino',
 'Burkinabe']

#### Step 3: Create an amedments dictionary

In [231]:
# create an amendments list
amendments = ['Egyptian',
              'Iranian',
              'Serbian',
              'British',
              'Czech',
              'South Korean',
              'Bosnian or Herzegovinian',
              'Canadian',
              'Tajikistani',
              'American',
              'British',
              'Spanish',
              'Philippine',
              'Burkinabé']

# create a dictionary of indexes and amendments
amendmentsDic = dict(zip(inconsistencies, amendments))

In [232]:
amendmentsDic

{'Coptic': 'Egyptian',
 'Persian': 'Iranian',
 'Yugoslav': 'Serbian',
 'Scottish': 'British',
 'Czechoslovakian': 'Czech',
 'Korean': 'South Korean',
 'Bosnian': 'Bosnian or Herzegovinian',
 'Canadian Inuit': 'Canadian',
 'Tajik': 'Tajikistani',
 'Native American': 'American',
 'Welsh': 'British',
 'Catalan': 'Spanish',
 'Filipino': 'Philippine',
 'Burkinabe': 'Burkinabé'}

#### Step 4: Applying amendments

In [233]:
# define a function to replace keys with values
def replaceInvalidNationaities(nlist,ndic):
    '''This function replaces invalid nationality values by a dictionary keys'''
    output = []
    for i in nlist:
        if i in ndic:
            output.append(ndic[i])
        else:
            output.append(i)
    return output

# apply the function on the nationality column in rawJson
rawJson['NationalityCleaner'] = rawJson['NationalityClean'].apply(replaceInvalidNationaities,ndic = amendmentsDic)

In [234]:
# did it work?
# create a list of nationalities from the countries data set
countriesNationalityList = countries['nationality'].to_list()

# create a list of nationalities from the rawJson data set
rawJsonNationalityList = rawJson['NationalityCleaner'].explode().unique()

# locate inconsistencies
inconsistencies = [i for i in rawJsonNationalityList if i not in countriesNationalityList]

# remove the first element (0)
inconsistencies.remove(inconsistencies[0])

inconsistencies

[]

### Adding a list with country codes for each artwork

In [235]:
# creating a countries dictionary
countriesDic = dict(zip(countries.index.values,countries.nationality))

In [236]:
# defining a new function
def addCountryList(cell,dictionary):
    '''This function iterates over a list of nationalities and returns a list of country codes'''
    output = []
    for i in cell:
        try:
            output.append(list(dictionary.keys())[list(dictionary.values()).index(i)])
        except:
            pass
    return output

# applying the function on the nationality column
rawJson['nationalityCodes'] = rawJson['NationalityCleaner'].apply(addCountryList,dictionary=countriesDic)

### Are there any invalid gender values?
As we can see in the next cell, the following gender values should be removed or changed in our data set:
- '' (empty string) - 1948 occurrences
- male - 30 occurrences (should be replaced with 'Male')
- female - 2 occurrences (should be replaced with 'Female')

In [237]:
# Gender
genderCheck = rawJson['Gender'].explode().value_counts()
print('Number of unique values: ',len(genderCheck))
print(genderCheck.to_string())

Number of unique values:  6
Male          120342
Female         21457
                1948
male              30
female             2
Non-Binary         1


In [238]:
# defining a function
def cleanGender(row):
    '''This function reads each element in the gender column, drops empty strings and returns first upper case letters for each element'''
    outputList = []
    for element in row:
        if element != '':
            outputList.append(element.title())
    return outputList

# add a new column
rawJson['GenderClean'] = rawJson['Gender'].apply(cleanGender)


In [239]:
# Did it work?
genderCheck = rawJson['GenderClean'].explode().value_counts()
print('Number of unique values: ',len(genderCheck))
print(genderCheck.to_string())

Number of unique values:  3
Male          120372
Female         21459
Non-Binary         1


### Are there any invalid artist values?
As we can see in the next cell, the following artist values should be removed or changed in our data set:
- Unknown photographer - 2896 occurrences
- Unknown Artist - 732 occurrences
- Unknown Designer - 460 occurrences
- Unknown Artists - 201 occurrences
- Various Unknown Artists - 73 occurrences
- unknown - 56 occurrences
- Designer unknown - 32 occurrences
- Unknown Director - 26 occurrences
- Unknown Author - 8 occurrences
- Unknown designer - 5 occurrences
- Unknown Graphic Artist - 3 occurrences
- Unknown sculptor - 1 occurrence


In [240]:
artistCheck = rawJson['Artist'].explode().value_counts()
print('Number of unique values: ',len(artistCheck))
print(artistCheck.to_string())

Number of unique values:  14628
Eugène Atget                                                                       5050
Louise Bourgeois                                                                   3382
Unknown photographer                                                               2896
Ludwig Mies van der Rohe                                                           2682
Jean Dubuffet                                                                      1436
Lee Friedlander                                                                    1338
Pablo Picasso                                                                      1322
Marc Chagall                                                                       1174
George Maciunas                                                                    1084
Henri Matisse                                                                      1069
Various Artists                                                                     977


In [241]:
# locate invalid values
invalidArtistValues = artistCheck.index[artistCheck.index.str.contains('nknown')].to_series()

In [242]:
# removing invalid artist values
rawJson['ArtistClean'] = rawJson['Artist'].apply(
    lambda row: [val for val in row if val not in invalidArtistValues]   
)

In [243]:
# did it work?
artistCheck = rawJson['ArtistClean'].explode().value_counts()
artistCheck.index[artistCheck.index.str.contains('nknown')]

Index([], dtype='object')

### Are there any invalid ThumbnailURL values?

In [244]:
# calling and applying the url function from the validators to validate the thumbnail urls
def urlValidator(value):
    '''This function validates url strings, and returns True for valid urls and False for invalid'''
    valueCheck = validators.url(str(value))
    if type(valueCheck) == validators.utils.ValidationFailure:
        return False
    else:
        return True
    
# adding a new column
rawJson['isValidThumbnailURL'] = rawJson.ThumbnailURL.apply(urlValidator)

# did it work?
rawJson['isValidThumbnailURL'].value_counts()

True     73969
False    65021
Name: isValidThumbnailURL, dtype: int64

In [245]:
# who are the invalid urls?
rawJson.loc[(rawJson['isValidThumbnailURL'] == False),['ThumbnailURL','isValidThumbnailURL']].sample(100)

,ThumbnailURL,isValidThumbnailURL
39040,None,False
30180,None,False
23717,None,False
102387,None,False
112951,None,False
...,...,...
16480,None,False
115219,None,False
35010,None,False
27559,None,False


### Are there any invalid artwork URL values?

In [246]:
# adding a new column
rawJson['isValidURL'] = rawJson.URL.apply(urlValidator)

# did it work?
rawJson['isValidURL'].value_counts()

True     84249
False    54741
Name: isValidURL, dtype: int64

## Reformatting the year column
We'll add a new column that represents the year when the artwork has been accomplished.

In [247]:
def tidyDate(value):
    '''this function reads a string that represents a range of years and returns the later year as an integer'''
    pattern = re.compile(r'[12][0789]\d\d[-–]?\d{0,4}|\d{1,2}[rt][dh]')
    p1 = re.compile(r'[12][0789]\d\d[-–][12][0789]\d\d')
    p2 = re.compile(r'[12][0789]\d\d[-–]\d\d\b')
    p3 = re.compile(r'[12][0789]\d\d[-–]\D')
    result = pattern.findall(str(value))
    output = []
    for i in result:
        if re.match(p1,i):
            output.append(int(i[-4:]))
        else:
            if re.match(p2,i):
                output.append(int(i[:2]+i[-2:]))
            else:     
                if re.match(p3,i):
                    output.append(int(i[:4]))
                else:
                    if len(i) == 4:
                        try:
                            output.append(int(i))
                        except:
                            pass
                    
    if len(output) == 0 or result == None:
        return 0
    else:
        return max(output)
    
# applying the function on the date column
rawJson['tidyDate'] = rawJson['Date'].apply(tidyDate)

In [248]:
# applying the function on the tidy date column
rawJson['Decade'] = (rawJson['tidyDate'].apply(lambda year: str(year)[:3]+'0' if len(str(year))==4 else 0)).astype('category')

In [249]:
rawJson['Decade'].value_counts()

1960    21229
1970    16163
1930    11389
1920    11173
1940    11046
2000    10888
1950    10827
1980    10754
1990    10185
1910     7024
1900     4869
2010     3956
0        3425
1890     2179
1860     1040
1870      887
1850      763
1880      531
1810      327
1840      216
1790       81
1820       20
1830       10
1760        5
1800        3
Name: Decade, dtype: int64

## Calculating the number of nationalities, artists and genders for each artwork

In [250]:
# using lambda functions
rawJson['numOfNationalities'] = rawJson['NationalityCleaner'].apply(lambda x: len(x))
rawJson['numOfArtists'] = rawJson['ArtistClean'].apply(lambda x: len(x))
rawJson['numOfGenders'] = rawJson['GenderClean'].apply(lambda x: len(x))

## Exporting a subset for valid artworks
Valid artworks have:
* no missing nationality values
* no missing artist names
* same number of elements in the nationality, artist and gender fields
* valid thumbnail url
* valid url
* valid date
* marked as curator approved

In [251]:
# setting the column for export
keptColumns = ['Title',
               'Artist',
               'NationalityCleaner',
               'nationalityCodes',
               'Classification',
               'tidyDate',
               'Decade',
               'GenderClean',
               'Medium',
               'ThumbnailURL',
               'ConstituentID',
               'ObjectID',
               'AccessionNumber',
               'URL']

# creating a subset
export = rawJson.loc[(rawJson['Cataloged'] == 'Y') &
                     (rawJson['tidyDate'] != 0) &
                     (rawJson['isValidURL'] == True) &
                     (rawJson['isValidThumbnailURL'] == True) &
                     (rawJson['numOfNationalities'] != 0) &
                     (rawJson['numOfArtists'] != 0) &
                     (rawJson['numOfGenders'] != 0) &
                     (rawJson['numOfArtists'] == rawJson['numOfNationalities']) &
                     (rawJson['numOfArtists'] == rawJson['numOfGenders']) &
                     (rawJson['numOfNationalities'] == rawJson['numOfGenders']),
                     keptColumns]

#Changing the index name
export.index.name = 'FID'

# reset the index
export = export.reset_index()

# print shape
print('The new subset contains ', str(export.shape[0]), 'artworks.')

# take a look
export.head()

The new subset contains  68320 artworks.


,FID,Title,Artist,NationalityCleaner,nationalityCodes,Classification,tidyDate,Decade,GenderClean,Medium,ThumbnailURL,ConstituentID,ObjectID,AccessionNumber,URL
0,0,"Ferdinandsbrücke Project, Vienna, Austria (Ele...",[Otto Wagner],[Austrian],[14],Architecture,1896,1890,[Male],Ink and cut-and-pasted painted pages on paper,http://www.moma.org/media/W1siZiIsIjU5NDA1Il0s...,[6210],2,885.1996,http://www.moma.org/collection/works/2
1,1,"City of Music, National Superior Conservatory ...",[Christian de Portzamparc],[French],[74],Architecture,1987,1980,[Male],Paint and colored pencil on print,http://www.moma.org/media/W1siZiIsIjk3Il0sWyJw...,[7470],3,1.1995,http://www.moma.org/collection/works/3
2,2,"Villa near Vienna Project, Outside Vienna, Aus...",[Emil Hoppe],[Austrian],[14],Architecture,1903,1900,[Male],"Graphite, pen, color pencil, ink, and gouache ...",http://www.moma.org/media/W1siZiIsIjk4Il0sWyJw...,[7605],4,1.1997,http://www.moma.org/collection/works/4
3,4,"Villa, project, outside Vienna, Austria, Exter...",[Emil Hoppe],[Austrian],[14],Architecture,1903,1900,[Male],"Graphite, color pencil, ink, and gouache on tr...",http://www.moma.org/media/W1siZiIsIjEyNiJdLFsi...,[7605],6,2.1997,http://www.moma.org/collection/works/6
4,30,"Memorial to the Six Million Jewish Martyrs, pr...",[Louis I. Kahn],[American],[231],Architecture,1968,1960,[Male],Charcoal and graphite on tracing paper,http://www.moma.org/media/W1siZiIsIjE3MyJdLFsi...,[2964],32,3.1997,http://www.moma.org/collection/works/32


# Exporting the subset to a JSON file

In [252]:
# export the artworks data frame
export.to_json('CleanedDataJG.json',orient='records')

In [254]:
# export the countries data frame
countriesExport = countries.iloc[:,0:7]
countriesExport.to_json('CleanCountries.json',orient = 'records')